In [13]:
import pandas as pd
import os

# Constants
QUESTION_TYPE = 1
ANSWER_TYPE = 2

file = os.path.join('..', 'data', 'pandas-code-dataset.csv')

dataset = pd.read_csv(file, encoding='ISO-8859-1', error_bad_lines=False, sep=";")
print('Amount of entries in the dataset = %d' % len(dataset))
dataset

Amount of entries in the dataset = 3205


,Unnamed: 0,AnswerCount,CommentCount,Id,ParentId,PostTypeId,Score,Tags,Title,Body,ViewCount,Code,CodeList,AllCode
0,0,2.0,7,7776679,NaN,1,25,<python><pandas>,append two data frame with pandas,<p>When I try to merge two dataframes by rows ...,25479.0,['bigdata = data1.append(data2)\nException: In...,"['bigdata = data1.append(data2)\n', 'Exception...","['bigdata = data1.append(data2)\n', 'Exception..."
1,1,7.0,3,7837722,NaN,1,187,<python><performance><for-loop><pandas>,What is the most efficient way to loop through...,<p>I want to perform my own complex operations...,195364.0,"[""Date,Open,High,Low,Close,Volume,Adj Close\n2...","['Date,Open,High,Low,Close,Volume,Adj Close\n2...","['Date,Open,High,Low,Close,Volume,Adj Close\n2..."
2,2,2.0,0,8916302,NaN,1,26,<python><csv><numpy><tab-delimited><pandas>,selecting across multiple columns with python ...,<p>I have a dataframe <code>df</code> in panda...,19805.0,"['df_greater_than10 = df[df[""colA""] > 10]\n']","['df_greater_than10 = df[df[""colA""] > 10]\n']","['df', 'pandas.read_table', 'colA', 'df_greate..."
3,3,3.0,16,8991709,NaN,1,134,<python><r><join><data.table><pandas>,Why are pandas merges in python faster than da...,"<p>I recently came across the <a href=""http://...",16173.0,[''],[],"['pandas', 'data.table', 'data.table', 'merge(..."
4,4,8.0,5,9555635,NaN,1,19,<python><numpy><scipy><enthought><pandas>,Open source Enthought Python alternative,<p>I used Enthought's python distribution as a...,6585.0,[''],[],"['numpy', 'scipy', 'sci-kits']"
5,5,2.0,2,9588331,NaN,1,21,<python><pandas>,Simple cross-tabulation in pandas,"<p>I stumbled across <a href=""http://pandas.py...",9493.0,"['AB,100.00\nAB,200.00\nAC,150.00\nAD,500.00\n...","['AB,100.00\nAB,200.00\nAC,150.00\nAD,500.00\n...","['pivot/crosstab/indexing', 'Panel', 'DataFram..."
6,6,3.0,0,9652832,NaN,1,38,<python><pandas><tsv>,How to I load a tsv file into a Pandas DataFrame?,<p>I'm new to python and pandas. I'm trying t...,32040.0,['>>> df1 = DataFrame(csv.reader(open(\'c:/~/t...,['>>> df1 = DataFrame(csv.reader(open(\'c:/~/t...,"['tsv', 'DataFrame', '>>> df1 = DataFrame(csv...."
7,7,6.0,0,9758450,NaN,1,34,<python><pandas>,Pandas convert dataframe to array of tuples,<p>I have manipulated some data using pandas a...,36433.0,['In [182]: data_set\nOut[182]: \n index data...,['In [182]: data_set\nOut[182]: \n index data...,['In [182]: data_set\nOut[182]: \n index data...
8,8,2.0,0,9762935,NaN,1,11,<python><pandas>,Add indexed column to DataFrame with pandas,"<p>I'm a beginning pandas user, and after stud...",8189.0,['df\n\n A ...,['df\n\n A ...,['df\n\n A ...
9,9,2.0,0,9787853,NaN,1,17,<python><pandas>,join or merge with overwrite in pandas,<p>I want to perform a join/merge/append opera...,6054.0,[''],[],"['df1', 'df2', 'df2', 'df2', 'df1', 'df1', 'df2']"


In [14]:
import re

text = """ Example of code to be filtered (only in In[11])
In [11]: df
df.append()
Out[11]: 
A B C D 
2000-01-03 -0.59885 -0.18141 -0.68828 -0.77572
2000-01-04 0.83935 0.15993 0.95911 -1.12959

In [12]: df[(df.values > 1.5).any(1)]
Out[12]: 
A B C D 
2000-01-05 2.8021 -0.1086 -1.62114 -0.2017
2000-01-06 0.7167 -0.2671 1.36029 1.7425
"""

def preprocessTerminalLikeCode(code):
    terminalRegex='In \[.+?\]:(.*?)Out\[.+?\]'
    sequence = re.compile(terminalRegex, re.DOTALL)
    terminalCode = sequence.findall(code)
    return [i.lstrip() for i in terminalCode]

In [15]:
import ast

text = """ Example of code to be filtered (only in In[11])
df
df.append()

A B C D 
2000-01-03 -0.59885 -0.18141 -0.68828 -0.77572
2000-01-04 0.83935 0.15993 0.95911 -1.12959

df[(df.values > 1.5).any(1)]
 
A B C D 
2000-01-05 2.8021 -0.1086 -1.62114 -0.2017
2000-01-06 0.7167 -0.2671 1.36029 1.7425
"""

def preprocessOutputCode(code):
    lines = code.split('\n')
    preprocessed = []
    for l in lines:
        try:
            ast.parse(l)
            preprocessed.append(l)
        except:
            # Ignore
            pass
    return '\n'.join(preprocessed)
            
# preprocessOutputCode(text)


In [16]:
import ast

def preprocessCode(code):
    # First Method for pre-processing
    terminalLikeCode = preprocessTerminalLikeCode(code)
    if terminalLikeCode:
        code = ''.join(terminalLikeCode)
        
    code = preprocessOutputCode(code)
    return code
    
def preprocessPost(post, col):
    strcodes = post[col]
    codes = ast.literal_eval(strcodes)
    processed = []
    for code in codes:
        processed.append(preprocessCode(code))
    post['ProcessedCode'] = processed
    return post


processed_dataset = dataset.apply(preprocessPost, axis=1, args=('Code',))


processed_dataset.to_csv(os.path.join('..', 'data', 'pandas-processedcode-dataset.csv'), encoding='ISO-8859-1', sep=';', doublequote=True)


7776679
7776679
7837722
8916302
8991709
9555635
9588331
9652832
9758450
9762935
9787853
9850954
10020591
10065051
10114399
10175068
10202570
10373660
10376647
10457584
10464738
10511024
10565282
10591000
10636024
10665889
10715519
10715965
10729210
10751127
10857924
10867028
10943478
10951341
10972410
10982089
11040626
11067027
11073609
11077023
11106823
11232275
11285613
11346283
11348183
11350770
11361985
11391969
11415701
11418192
11495051
11548005
11587782
11615504
11622652
11637384
11640243
11697887
11707586
11707805
11714768
11728836
11811392
11858472
11869910
11881165
11927715
11941492
11976503
12021730
12021754
12047193
12052067
12065885
12096252
12100497
12101113
12133075
12152716
12168648
12169170
12182744
12190874
12200693
12203901
12207326
12250024
12278347
12286607
12307099
12322289
12322779
12329853
12356501
12358360
12376863
12389898
12406162
12433076
12436979
12497402
12504493
12504951
12504976
12525722
12555323
12569730
12589481
12604909
12625650
12680754
12704305
1272

27673231
27759084
27787930
27842613
27884268
27896214
27905295
27975069
28006793
28009370
28017091
28135436
28142420
28161356
28199524
28218698
28219902
28227612
28228781
28236305
28259301
28272137
28293028
28311655
28371308
28384680
28417293
28465633
28538536
28590663
28595701
28651079
28661258
28676916
28679930
28757389
28773342
28901683
28931224
28986489
29177498
29206612
29226210
29233283
29241836
29287224
29314033
29334463
29370057
29370211
29432629
29459461
29464234
29498652
29516084
29517072
29525808
29530232
29532894
29545704
29550414
29576430
29607222
29655111
29706278
29763620
29765548
29794959
29815129
29836836
29902714
29919306
29954263
29971075
30009948
30017491
30023927
30026815
30053329
30059260
30088006
30102232
30132282
30133280
30211923
30222533
30272300
30317119
30327417
30328646
30357276
30462807
30519487
30522724
30522982
30530663
30627968
30631325
30651724
30653642
30781037
30787901
30808430
30922213
30925079
30926670
31029560
31037298
31247198
31329627
31331358
3

19599776
19600533
19603918
19609945
19609954
19611857
19619020
19632099
19633103
19726078
19736406
19739768
19758398
19782137
19791302
19798528
19809616
19818942
19819118
19821311
19828967
19851521
19861545
19867768
19895152
19900276
19913845
19915115
19918849
19922732
19928288
19937902
19960116
19960136
19961557
19961872
19966142
19969224
19973722
19976286
19982756
19991632
19996208
20006954
20012628
20015080
20019449
20024879
20027386
20032254
20033218
20033232
20038973
20039057
20043785
20051631
20059818
20067665
20069379
20076611
20084843
20084895
20096494
20096827
20107825
20120225
20154429
20159305
20167984
20168416
20181686
20199798
20200594
20206825
20221655
20228113
20230859
20231632
20233649
20235451
20250947
20250996
20297639
20301769
20304311
20312816
20333894
20334902
20341058
20375692
20384317
20410720
20444256
20455090
20459839
20461206
20461929
20481080
20491748
20496583
20523271
20557179
20566408
20574460
20603020
20612691
20619971
20627316
20637559
20638258
20639234
2

33259038
33271634
33324058
33346694
33374834
33375383
33381151
33381246
33387356
33404243
33480745
33555435
33570065
33577649
33590284
33687073
33763855
33768634
33777664
33780558
33782239
33782409
33786696
33795876
33798922
33814054
33837592
33872824
33873209
33913961
33952294
33957850
33967359
33986975
33993078
33997632
34013098
34082664
34092032
34094058
34097939
34110546
34130874
34192820
34262133
34272155
34277514
34282362
34297689
34311080
34333886
34365537
34402423
34457902
34466473
34530065
34531543
34548894
34551914
34555201
34576537
34592295
34614046
34683105
34687479
34756965
34790248
34794112
34812293
34863702
34879805
34883876
34896797
34896835
34897294
34916691
34962199
34962592
34996876
35068123
35087831
35106735
35138807
35139819
35203149
35203658
35212740
35219658
35240942
35245297
35246041
35282530
35318270
35368792
35385805
35387028
35387129
35415751
35416021
35446404
35453836
35476539
35509134
35523946
35531218
35583219
35616082
35619846
35715029
35768306
35783766
3